In [1]:
import os
import itertools
import weakref
from typing import Any, Dict, List, Set
import logging
from collections import OrderedDict



import torch
from fvcore.nn.precise_bn import get_bn_modules
from MOT_dataset import get_MOT17_train, get_MOT17_test, MOT_CLASS_NAMES
import detectron2.utils.comm as comm
from detectron2.utils.logger import setup_logger
from detectron2.checkpoint import DetectionCheckpointer
from detectron2.config import get_cfg
from detectron2.data import build_detection_train_loader
from detectron2.engine import DefaultTrainer, default_argument_parser, default_setup, launch, create_ddp_model, \
    AMPTrainer, SimpleTrainer, hooks
from detectron2.evaluation import COCOEvaluator, LVISEvaluator, verify_results
from detectron2.solver.build import maybe_add_gradient_clipping
from detectron2.modeling import build_model
from detectron2.data import DatasetCatalog, MetadataCatalog


from diffusiondet.predictor import VisualizationDemo

from diffusiondet import DiffusionDetDatasetMapper, add_diffusiondet_config, DiffusionDetWithTTA
from diffusiondet.util.model_ema import add_model_ema_configs, may_build_model_ema, may_get_ema_checkpointer, EMAHook, \
    apply_model_ema_and_restore, EMADetectionCheckpointer

import argparse
from train_mot17 import Trainer


In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [3]:
print(device)

cuda


In [4]:
def setup(args):
    """
    Create configs and perform basic setups.
    """
    cfg = get_cfg()
    add_diffusiondet_config(cfg)
    add_model_ema_configs(cfg)
    cfg.merge_from_file(args.config_file)
    cfg.merge_from_list(args.opts)
    cfg.freeze()
    default_setup(cfg, args)
    return cfg

In [5]:
args = argparse.Namespace()
args.num_gpus = 1
args.config_file = 'configs/diffdet.mot17.swinbase.yaml'
args.eval_only = True
args.opts = ['MODEL.WEIGHTS', 'models/diffdet_coco_swinbase.pth']

cfg = setup(args)
#trainer = Trainer(cfg)

[01/12 15:22:29 detectron2]: Rank of current process: 0. World size: 1
[01/12 15:22:30 detectron2]: Environment info:
----------------------  -------------------------------------------------------------------------------
sys.platform            linux
Python                  3.7.12 | packaged by conda-forge | (default, Oct 26 2021, 06:08:21) [GCC 9.4.0]
numpy                   1.21.5
detectron2              0.6 @/opt/conda/lib/python3.7/site-packages/detectron2
Compiler                GCC 7.3
CUDA compiler           CUDA 11.3
detectron2 arch flags   3.7, 5.0, 5.2, 6.0, 6.1, 7.0, 7.5, 8.0, 8.6
DETECTRON2_ENV_MODULE   <not set>
PyTorch                 1.10.0 @/opt/conda/lib/python3.7/site-packages/torch
PyTorch debug build     False
GPU available           Yes
GPU 0                   Tesla T4 (arch=7.5)
Driver version          510.47.03
CUDA_HOME               /usr/local/cuda
Pillow                  9.2.0
torchvision             0.11.0 @/opt/conda/lib/python3.7/site-packages/torchvision


In [ ]:
demo = VisualizationDemo(cfg)

In [ ]:

def compute_iou(bbox1, bbox2):
  #Returns the ratio of the intersection over the union ox the two boxes

  # Compute the coordinates of the intersection rectangle
  x_left = max(bbox1[0], bbox2[0])
  y_top = max(bbox1[1], bbox2[1])
  x_right = min(bbox1[2], bbox2[2])
  y_bottom = min(bbox1[3], bbox2[3])

  # Check if the intersection is valid
  if x_right < x_left or y_bottom < y_top:
      return 0.0

  # Compute the area of the intersection
  intersection_area = (x_right - x_left) * (y_bottom - y_top)

  # Compute the areas of the bounding boxes
  bbox1_area = (bbox1[2] - bbox1[0]) * (bbox1[3] - bbox1[1])
  bbox2_area = (bbox2[2] - bbox2[0]) * (bbox2[3] - bbox2[1])

  # Compute the IoU
  iou = intersection_area / (bbox1_area + bbox2_area - intersection_area)
  return iou

In [6]:
dataloader = Trainer.build_train_loader(cfg)

[01/12 15:23:06 d2.data.build]: Distribution of instances among all 13 categories:
|  category  | #instances   |   category    | #instances   |   category    | #instances   |
|:----------:|:-------------|:-------------:|:-------------|:-------------:|:-------------|
|   other    | 0            |  pedestrian   | 112297       | person on v.. | 1864         |
|    car     | 6189         |    bicycle    | 13527        |   Motorbike   | 1050         |
| NMVehicle  | 11           | Static person | 12041        |  Distractor   | 3595         |
|  occluder  | 3724         | occluder_gr.. | 28573        | occluder_full | 19580        |
| reflection | 2250         |               |              |               |              |
|   total    | 204701       |               |              |               |              |
[01/12 15:23:06 d2.data.build]: Using training sampler TrainingSampler
[01/12 15:23:06 d2.data.common]: Serializing 5316 elements to byte tensors and concatenating them all ...
[01/

In [7]:
model = Trainer.build_model(cfg)

/opt/conda/lib/python3.7/site-packages/torch/functional.py:445: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  /opt/conda/conda-bld/pytorch_1634272168290/work/aten/src/ATen/native/TensorShape.cpp:2157.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [8]:
for idx, test in enumerate(dataloader):
    output = model(test)
    print(output)
    
    if idx == 0:
        break
    


/opt/conda/lib/python3.7/site-packages/detectron2/structures/image_list.py:88: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  max_size = (max_size + (stride - 1)) // stride * stride


{'loss_ce': tensor(2.2414, device='cuda:0', grad_fn=<MulBackward0>), 'loss_bbox': tensor(3.5885, device='cuda:0', grad_fn=<MulBackward0>), 'loss_giou': tensor(2.4817, device='cuda:0', grad_fn=<MulBackward0>), 'loss_ce_0': tensor(2.3419, device='cuda:0', grad_fn=<MulBackward0>), 'loss_bbox_0': tensor(11.5281, device='cuda:0', grad_fn=<MulBackward0>), 'loss_giou_0': tensor(2.3156, device='cuda:0', grad_fn=<MulBackward0>), 'loss_ce_1': tensor(2.1665, device='cuda:0', grad_fn=<MulBackward0>), 'loss_bbox_1': tensor(7.8298, device='cuda:0', grad_fn=<MulBackward0>), 'loss_giou_1': tensor(2.4736, device='cuda:0', grad_fn=<MulBackward0>), 'loss_ce_2': tensor(2.0351, device='cuda:0', grad_fn=<MulBackward0>), 'loss_bbox_2': tensor(3.7315, device='cuda:0', grad_fn=<MulBackward0>), 'loss_giou_2': tensor(2.5963, device='cuda:0', grad_fn=<MulBackward0>), 'loss_ce_3': tensor(2.4476, device='cuda:0', grad_fn=<MulBackward0>), 'loss_bbox_3': tensor(3.2716, device='cuda:0', grad_fn=<MulBackward0>), 'loss_

In [ ]:
def object_tracking_frame_txt_writing(frames_folder,video_name,detection_file_name, reverse=False,video_output=True):

  #Load the file of interest : (worked on this one because it is one of the shortest of the MOT17)
  frames_dir = "/content/drive/MyDrive/DiffusionDet/diffusiondet/videos/"+ frames_folder
  frames_files_list = [f for f in os.listdir(frames_dir) if f.endswith(".jpg")]
  frames_files_list.sort(reverse=reverse)
  
  #We initialise the frame number
  frame_number = 1
  
  # Initialize the dictionary of track IDs
  tracks = {}
  track_ids = {}

  #We initialise the detection function
  detections_list = []

  #We initialize the output video
  if video_output==True:
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    video_path = os.path.join("/content/drive/MyDrive/DiffusionDet/diffusiondet/tracked_videos", video_name)
    frame_0 = cv2.imread(frames_dir + '/' + frames_files_list[0])
    out = cv2.VideoWriter(video_path, fourcc, 30, (frame_0.shape[1], frame_0.shape[0]))

  #We intialize the csv files that writes the boxes detected
  with open('/content/drive/MyDrive/DiffusionDet/diffusiondet/tracked_videos/'+ detection_file_name, 'w', newline='') as txtfile:
    
    # Iterate through frames in the video
    for i,frame_name in enumerate(frames_files_list):
      frame = cv2.imread(frames_dir+'/'+frame_name)
      print(frame_name)

      if frame_number not in tracks:
          tracks[frame_number] = []

      print(str(frame_number) + '/' +str(len(frames_files_list)))

      detections = demo.run_on_image(frame)
      # Format the detections for convenience
      # Replace this with your own code for formatting the detections
      bboxes = detections[0]['instances']._fields['pred_boxes'].tensor
      labels = detections[0]['instances']._fields['pred_classes']
      scores = detections[0]['instances']._fields['scores']
      keep = nms(bboxes, scores, 0.5)
      bboxes = bboxes[keep]
      scores = scores[keep]
      labels = labels[keep]

      for bbox, label, score in zip(bboxes, labels, scores):
          # Initialize a new track
          track = {'bbox': bbox, 'conf': score}

          # Find the closest track in the previous frame based on the IoU
          max_iou = 0
          best_match = None
          if frame_number > 1 and tracks[frame_number - 1]:
              for previous_track in tracks[frame_number - 1]:
                  iou = compute_iou(bbox, previous_track['bbox'])
                  if iou > max_iou:
                      max_iou = iou
                      best_match = previous_track
                      
          if max_iou > 0.5:
              # Update the track with the new bounding box and confidence
              best_match['bbox'] = bbox
              best_match['conf'] = score
              if 'id' in track:  # check if the track already has an ID
                  track['id'] = track['id']  # use the existing ID
              else:
                  track['id'] = best_match['id']  # assign the ID of the best matching track from the previous frame
          else:
              if track_ids:  # check if the dictionary of track IDs is not empty
                  # Assign a new ID that hasn't been used yet
                  unused_ids = [id for id in track_ids if id not in track_ids.values()]
                  if unused_ids:  # check if there are any unused IDs
                      track['id'] = unused_ids[0]
                      track_ids.update({track['id']: track['id']})  # add the new ID to the dictionary of track IDs
                  else:
                      track['id'] = max(track_ids) + 1  # assign a new ID
                      track_ids.update({track['id']: track['id']})  # add the new ID to the dictionary of track IDs
              else:
                  track['id'] = 1
                  track_ids.update({track['id']: track['id']})  # add the new ID to the dictionary of track IDs
          tracks[frame_number].append(track)  # add the track to the list of tracks

          #Now we write each boxes in the file
          #frame, id, bb_left, bb_top, bb_width, bb_height, conf, x, y, z
          bbox_cpu = bbox.to(torch.device('cpu')).numpy()
          label_cpu = label.to(torch.device('cpu')).numpy()
          score_cpu = score.to(torch.device('cpu')).numpy()
          width_bbox = np.abs(bbox_cpu[0]-bbox_cpu[2])
          height_bbox = np.abs(bbox_cpu[1]-bbox_cpu[3])
          if(reverse==False):
            txtfile.write(str(frame_number)+','+str(label_cpu)+','+str(bbox_cpu[0])+','+str(bbox_cpu[1])+','+str(width_bbox)+','+str(height_bbox)+','+str(score_cpu)+ '\n')
            detections_list.append([frame_number,int(label_cpu),bbox_cpu[0],bbox_cpu[1],width_bbox,height_bbox,float(score_cpu)])
          if(reverse==True):
            txtfile.write(str(len(frames_files_list)-frame_number+1)+','+str(label_cpu)+','+str(bbox_cpu[0])+','+str(bbox_cpu[1])+','+str(width_bbox)+','+str(height_bbox)+','+str(score_cpu)+ '\n')
            detections_list.append([len(frames_files_list)-frame_number+1,int(label_cpu),bbox_cpu[0],bbox_cpu[1],width_bbox,height_bbox,float(score_cpu)])
      #We only create a video output if required
      if video_output==True:
        colors = [(255, 0, 0), (0, 255, 0), (0, 0, 255), (255, 255, 0), (255, 0, 255), (0, 255, 255),(128, 0, 0), (0, 128, 0), (0, 0, 128), (128, 128, 0), (128, 0, 128), (0, 128, 128),           (64, 0, 0), (0, 64, 0), (0, 0, 64)]
        for track in tracks[frame_number]:
            xmin, ymin, xmax, ymax = track['bbox']
            color = colors[track['id'] % len(colors)]
            cv2.rectangle(frame, (int(xmin), int(ymin)), (int(xmax), int(ymax)), color, 2)
            cv2.putText(frame, str(track['id']) + ' ' + str("{:.2f}".format(track['conf'].item())), (int(xmin), int(ymin) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
        out.write(frame)

      frame_number += 1
  if(video_output==True):
    out.release() 
  return detections_list